Определение спам сообщений

In [101]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, precision_recall_curve
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\79159\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [78]:
df = pd.read_csv("labeled.csv", sep = ',')

In [79]:
df.shape

(14412, 2)

In [80]:
df.head(10)

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
5,Для каких стан является эталоном современная с...,1.0
6,В шапке были ссылки на инфу по текущему фильму...,0.0
7,УПАД Т! ТАМ НЕЛЬЗЯ СТРОИТЬ! ТЕХНОЛОГИЙ НЕТ! РА...,1.0
8,"Ебать тебя разносит, шизик.\n",1.0
9,"Обосрался, сиди обтекай\n",1.0


In [81]:
df["toxic"] = df["toxic"].apply(int)

In [82]:
df.head(5)

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1
1,"Хохлы, это отдушина затюканого россиянина, мол...",1
2,Собаке - собачья смерть\n,1
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1


In [83]:
df['toxic'].value_counts()

toxic
0    9586
1    4826
Name: count, dtype: int64

In [84]:
for c in df[df["toxic"] == 1]['comment'].head(5):
    print(c)

Верблюдов-то за что? Дебилы, бл...

Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.

Собаке - собачья смерть

Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?

тебя не убедил 6-страничный пдф в том, что Скрипалей отравила Россия? Анализировать и думать пытаешься? Ватник что ли?)



In [85]:
for c in df[df["toxic"] == 0]['comment'].head(5):
    print(c)

В шапке были ссылки на инфу по текущему фильму марвел. Эти ссылки были заменены на фразу Репортим брипидора, игнорируем его посты. Если этого недостаточно, чтобы понять, что модератор абсолютный неадекват, и его нужно лишить полномочий, тогда эта борда пробивает абсолютное дно по неадекватности.

Почитайте посты у этого автора,может найдете что нибудь полезное. Надеюсь помог) https: pikabu.ru story obyichnyie budni dezsluzhbyi 4932098

Про графику было обидно) я так то проходил все серии гта со второй части по пятую, кроме гта 4. И мне не мешала графика ни в одной из частей. На компе у меня было куча видеокарт. Начиная с 32мб RIVA TNT и заканчивая 2Гб 560Ti на которой я спокойно играю который год в танки, гта5, ведьмака3 купил на распродаже и начал проходить. Да, не на ультрах. С пониженными текстурами. И не мешает. Я не понимаю дрочева на графике, требовать графику уровня плойки 4 минимум. Мне надо чтобы глаза не резало, только и всего. По поводу управления, мне не хватает переходника

In [86]:
train_df, test_df, = train_test_split(df, test_size=500)

In [87]:
test_df.shape

(500, 2)

In [88]:
test_df['toxic'].value_counts()

toxic
0    335
1    165
Name: count, dtype: int64

In [89]:
sentence_example = df.iloc[1]["comment"]
tokens = word_tokenize(sentence_example, language="russian")
tokens_without_punc = [i for i in tokens if i not in string.punctuation]
rus_stop_word = stopwords.words("russian")
tokens_without_punc_and_stopwords = [i for i in tokens_without_punc if i not in rus_stop_word]
snowball = SnowballStemmer(language = 'russian')
stemmed_tokens = [snowball.stem(i) for  i in tokens_without_punc_and_stopwords]

In [90]:
print("ичходный текст - ", sentence_example)
print("tokens - ", tokens)
print("tokens_without_punc - ", tokens_without_punc)
print("tokens_without_punc_and_stopwords - ", tokens_without_punc_and_stopwords)
print("stemmed_tokens - ", stemmed_tokens)


ичходный текст -  Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.

tokens -  ['Хохлы', ',', 'это', 'отдушина', 'затюканого', 'россиянина', ',', 'мол', ',', 'вон', ',', 'а', 'у', 'хохлов', 'еще', 'хуже', '.', 'Если', 'бы', 'хохлов', 'не', 'было', ',', 'кисель', 'их', 'бы', 'придумал', '.']
tokens_without_punc -  ['Хохлы', 'это', 'отдушина', 'затюканого', 'россиянина', 'мол', 'вон', 'а', 'у', 'хохлов', 'еще', 'хуже', 'Если', 'бы', 'хохлов', 'не', 'было', 'кисель', 'их', 'бы', 'придумал']
tokens_without_punc_and_stopwords -  ['Хохлы', 'это', 'отдушина', 'затюканого', 'россиянина', 'мол', 'вон', 'хохлов', 'хуже', 'Если', 'хохлов', 'кисель', 'придумал']
stemmed_tokens -  ['хохл', 'эт', 'отдушин', 'затюкан', 'россиянин', 'мол', 'вон', 'хохл', 'хуж', 'есл', 'хохл', 'кисел', 'придума']


In [91]:
rus_stop_word = stopwords.words("russian")
snowball = SnowballStemmer(language = 'russian')

def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language="russian")
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in rus_stop_word]
    tokens = [snowball.stem(i) for i in tokens]


    return tokens

In [92]:
tokenize_sentence(sentence_example)

['хохл',
 'эт',
 'отдушин',
 'затюкан',
 'россиянин',
 'мол',
 'вон',
 'хохл',
 'хуж',
 'есл',
 'хохл',
 'кисел',
 'придума']

In [93]:
vectorizer =TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True))

In [94]:
features = vectorizer.fit_transform(train_df["comment"])

d:\учеба\train_summer\venv\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [95]:
model = LogisticRegression(random_state= 0)
model.fit(features, train_df["toxic"])

LogisticRegression(random_state=0)

In [102]:
model1 = SVC(random_state= 0)
model1.fit(features, train_df["toxic"] )

SVC(random_state=0)

In [104]:
model1_pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True))),
    ("model1", SVC(random_state = 0))
])

In [105]:
model1_pipeline.fit(train_df["comment"], train_df["toxic"])

d:\учеба\train_summer\venv\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x000001BA56883920>)),
                ('model1', SVC(random_state=0))])

In [99]:
recall_score(y_true=test_df["toxic"], y_pred= model1_pipeline.predict(test_df["comment"]))

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [ ]:
model.predict(features[2])

array([0])

In [ ]:
train_df["comment"].iloc[2]

'прошу прощенья, вы видели фото из коридора? О какой чрезмерной нагрузки вы говорите? На весь дом? Там проводка алюминиевая у который давным давно вышел срок годности.\n'

In [ ]:
model_pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True)) ),
    ("model", LogisticRegression(random_state= 0) )
])

In [ ]:
model_pipeline.fit(train_df["comment"], train_df["toxic"])

d:\учеба\train_summer\venv\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x000001BA52EEE0C0>)),
                ('model', LogisticRegression(random_state=0))])

In [ ]:
model_pipeline.predict(["привет, у меня все нормально   "])

array([0])

In [ ]:
precision_score(y_true=test_df["toxic"], y_pred= model_pipeline.predict(test_df["comment"]))

np.float64(0.9148936170212766)

In [ ]:
recall_score(y_true=test_df["toxic"], y_pred= model_pipeline.predict(test_df["comment"]))


np.float64(0.6898395721925134)

In [ ]:
prec, rec, thresholds = precision_recall_curve(y_true=test_df["toxic"], probas_pred=model_pipeline.predict_proba(test_df["comment"])[:,1])

d:\учеба\train_summer\venv\Lib\site-packages\sklearn\metrics\_ranking.py:993: FutureWarning: probas_pred was deprecated in version 1.5 and will be removed in 1.7.Please use ``y_score`` instead.
  warnings.warn(


In [ ]:
np.where(prec > 0.95)

(array([398, 399, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423,
        424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436,
        437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449,
        450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462,
        463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475,
        476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488,
        489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500]),)

In [ ]:
precision_score(y_true=test_df["toxic"], y_pred= model_pipeline.predict_proba(test_df["comment"])[:,1] > thresholds[398])

np.float64(0.9504950495049505)

In [ ]:
recall_score(y_true=test_df["toxic"], y_pred= model_pipeline.predict_proba(test_df["comment"])[:,1] > thresholds[398])

np.float64(0.5133689839572193)